# Eat Safe, Love

## Notebook Set Up

In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# assign the uk_food database to a variable name
db = mongo["uk_food"]

In [5]:
# review the collections in our database
db.list_collection_names()

['establishments']

In [6]:
# assign the collection to a variable
establishments_c = db["establishments"]

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [19]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": {"$eq": 20}}

# Use count_documents to display the number of documents in the result
results = list(establishments_c.find(query))
print(f'There are {establishments_c.count_documents(query)} restaurants with a hygiene score of 20. \n')

# Display the first document in the results using pprint
pprint(establishments_c.find_one(query))

There are 41 restaurants with a hygiene score of 20. 

{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63ef301681e5f0fbe1b6e67a'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}

In [21]:
# Convert the result to a Pandas DataFrame
hygiene_20_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(hygiene_20_df.shape) # rows -> Columns (41 rows)

# Display the first 10 rows of the DataFrame -> Higher hygiene scores is bad so it fits!
hygiene_20_df[["BusinessName", "RatingValue", "scores"]].head(10) 

(41, 28)


,BusinessName,RatingValue,scores
0,The Chase Rest Home,0,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceI..."
1,Brenalwood,0,"{'Hygiene': 20, 'Structural': 15, 'ConfidenceI..."
2,Melrose Hotel,0,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceI..."
3,Seaford Pizza,1,"{'Hygiene': 20, 'Structural': 10, 'ConfidenceI..."
4,Golden Palace,1,"{'Hygiene': 20, 'Structural': 10, 'ConfidenceI..."
5,Ashby's Butchers,0,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceI..."
6,South Sea Express Cuisine,0,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceI..."
7,Golden Palace,0,"{'Hygiene': 20, 'Structural': 15, 'ConfidenceI..."
8,The Tulip Tree,1,"{'Hygiene': 20, 'Structural': 5, 'ConfidenceIn..."
9,F & S,0,"{'Hygiene': 20, 'Structural': 20, 'ConfidenceI..."


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [9]:
# Since some restaurants are 'awaiting inspection', this field is a string
## Find all restaurants that are already rated -> convert those to integer -> query them

query = {"RatingValue": {"$type": "string", "$regex": "^[0-9]+$"}}
update = [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
establishments_c.update_many(query, update)


In [10]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {"$regex": "London"}, 
         "RatingValue": {"$gte": 4}
        }

# Use count_documents to display the number of documents in the result
results = list(establishments_c.find(query))
print(f'There are {establishments_c.count_documents(query)} restaurants in London with a rating score of 4 or over. \n')

# Display the first document in the results using pprint
pprint(establishments_c.find_one(query))

There are 33 restaurants in London with a rating score of 4 or over. 

{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63ef301881e5f0fbe1b70095'),
 'geocode': {'latitude': 51.369321, 'longitude': 0.508551},
 'links': [{'href': 'https://api.

In [11]:
# Convert the result to a Pandas DataFrame - interestingly json_normalize is missing 1 restaurant in the results (the first)
good_london_rest_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame 
print(good_london_rest_df["RatingValue"].count())

# Display the first 10 rows of the DataFrame
good_london_rest_df[["BusinessName", "RatingValue", "scores"]].head(10)

33


,BusinessName,RatingValue,scores
0,Charlie's,4,"{'Hygiene': 5, 'Structural': 10, 'ConfidenceIn..."
1,Mv City Cruises Erasmus,5,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM..."
2,Benfleet Motor Yacht Club,4,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM..."
3,Coombs Catering t/a The Lock and Key,5,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM..."
4,Tilbury Seafarers Centre,5,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM..."
5,Mv Valulla,5,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM..."
6,Tereza Joanne,5,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM..."
7,Brick Lane Brews,4,"{'Hygiene': 10, 'Structural': 5, 'ConfidenceIn..."
8,The Nuance Group (UK) Limited,5,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM..."
9,WH Smith,5,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM..."


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [12]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {"geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
         "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
         "RatingValue": 5}

sort = [("scores.Hygiene", 1)]

# Print the results
results = list(establishments_c.find(query).sort(sort))
print(f'There are {establishments_c.count_documents(query)} restaurants near Penang Flavours with a rating score of 5. \n')
pprint(establishments_c.find_one(query))

There are 87 restaurants near Penang Flavours with a rating score of 5. 

{'AddressLine1': '152 Plumstead High Street',
 'AddressLine2': '',
 'AddressLine3': 'Plumstead',
 'AddressLine4': 'Greenwich',
 'BusinessName': 'TIWA N TIWA African Restaurant Ltd',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4646.930146257832,
 'FHRSID': 1069652,
 'LocalAuthorityBusinessID': 'PI/000206841',
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'SE18 1JQ',
 'RatingDate': '2020-10-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63ef301b81e5f0fbe1b73e85'),
 'geocode': {'latitude': 51.4870351, 'longitude': 0.0927429},
 'links': [{'href': 'http://api.ratings.food.gov.uk/establi

In [13]:
# Convert result to Pandas DataFrame
good_restaurants_near_penang_df = pd.DataFrame(results)

print(good_restaurants_near_penang_df.shape) # 87 rows

good_restaurants_near_penang_df.head(10)

(87, 28)


,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links
0,63ef301b81e5f0fbe1b73eac,695223,0,PI/000178842,Iceland,Retailers - supermarkets/hypermarkets,7840,144 - 146 Plumstead High Street,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0924199968576431, 'latitude': ...",,4646.946071,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
1,63ef301b81e5f0fbe1b73ebc,1380578,0,14425,Howe and Co Fish and Chips - Van 17,Mobile caterer,7846,Restaurant And Premises 107A Plumstead High St...,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.0925370007753372, 'latitude': ...",,4646.955931,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
2,63ef301b81e5f0fbe1b73ed9,694609,0,PI/000116619,Volunteer,Pub/bar/nightclub,7843,130 - 132 Plumstead High Street,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.09208, 'latitude': 51.4873437}",,4646.965635,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
3,63ef301b81e5f0fbe1b73ef1,695241,0,PI/000179088,Plumstead Manor Nursery,Caring Premises,5,Plumstead Manor School Old Mill Road,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.0859939977526665, 'latitude': ...",,4646.974010,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
4,63ef301b81e5f0fbe1b73ef4,694478,0,PI/000086506,Atlantic Fish Bar,Takeaway/sandwich shop,7844,35 Lakedale Road,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.0912164, 'latitude': 51.4867296}",,4646.974612,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
5,63ef301b81e5f0fbe1b73ef6,909912,0,PI/000201100,Lumbini Grocery Ltd T/A Al-Iman,Retailers - other,4613,17 Lakedale Road,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0916256, 'latitude': 51.4871632}",,4646.975587,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
6,63ef301b81e5f0fbe1b73f14,694482,0,PI/000086551,Greggs,Takeaway/sandwich shop,7844,6 Lakedale Road,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.0912820026278496, 'latitude': ...",,4646.987290,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
7,63ef301b81e5f0fbe1b73f15,695204,0,PI/000177985,Tesco,Retailers - supermarkets/hypermarkets,7840,18 Lakedale Road,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0912820026278496, 'latitude': ...",,4646.987290,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
8,63ef301b81e5f0fbe1b73f3c,1211907,0,13217,Dosa & Sambal Express,Restaurant/Cafe/Canteen,1,107A Plumstead High Street,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.091566, 'latitude': 51.487757

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [14]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_query = {"$match": {"scores.Hygiene" : {"$eq" : 0 }}}

group_query = {'$group': {'_id': {"LocalAuthorityName": "$LocalAuthorityName"}, 
                          'count': { '$sum': 1 }
                         }
              }

sort_query = {"$sort": {"count" : -1}}

pipeline = [match_query, group_query, sort_query]
# Print the number of documents in the result
results = list(establishments_c.aggregate(pipeline))

# Print the first 10 results
pprint(results[0:10])

[{'_id': {'LocalAuthorityName': 'Thanet'}, 'count': 1130},
 {'_id': {'LocalAuthorityName': 'Greenwich'}, 'count': 882},
 {'_id': {'LocalAuthorityName': 'Maidstone'}, 'count': 713},
 {'_id': {'LocalAuthorityName': 'Newham'}, 'count': 711},
 {'_id': {'LocalAuthorityName': 'Swale'}, 'count': 686},
 {'_id': {'LocalAuthorityName': 'Chelmsford'}, 'count': 680},
 {'_id': {'LocalAuthorityName': 'Medway'}, 'count': 672},
 {'_id': {'LocalAuthorityName': 'Bexley'}, 'count': 607},
 {'_id': {'LocalAuthorityName': 'Southend-On-Sea'}, 'count': 586},
 {'_id': {'LocalAuthorityName': 'Tendring'}, 'count': 542}]


In [15]:
# Convert the result to a Pandas DataFrame
good_hygiene_df = pd.json_normalize(results)
# Display the number of rows in the DataFrame
print(good_hygiene_df.shape)

# Display the first 10 rows of the DataFrame - Changing column name + reordering column (count appeared first)
good_hygiene_df = good_hygiene_df.rename(columns={'_id.LocalAuthorityName':'local_authority_name'})
good_hygiene_df = good_hygiene_df.reindex(columns=['local_authority_name','count'])
good_hygiene_df.head(10)

(55, 2)


,local_authority_name,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
